In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder, StandardScaler

import joblib

In [2]:
# -----------------------------------------------------------
# Load dataset
# -----------------------------------------------------------
df = pd.read_csv(r"D:\project\smart_premium\data\train.csv", index_col="id")
df.drop_duplicates(inplace=True)

In [3]:
df

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,NaN,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,NaN,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,0.0,19.0,NaN,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0


In [4]:
# -----------------------------------------------------------
# Independent and Dependent features
# -----------------------------------------------------------
       
target = "Premium Amount"
x = df.drop(target, axis=1)             #Independent features
y = df[target]                          #Dependent features

In [76]:
# -----------------------------------------------------------
# Identify numerical and categorical columns
# -----------------------------------------------------------

num_cols = x.select_dtypes(include=["float64", "int64"]).columns
cat_cols = x.select_dtypes(include=["object"]).columns

In [5]:
cat_cols = x.select_dtypes(include="object").columns
num_cols = x.select_dtypes(exclude="object").columns


In [6]:
# -----------------------------------------------------------
# Preprocessing pipeline
# -----------------------------------------------------------

num_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),      #Median imputer for numerical features
    ("scaler", StandardScaler())])                      #StandardScaler for numerical features


cat_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),   #Most frequent imputer for categorical features
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value",
                                unknown_value=-1))])

preprocessor = ColumnTransformer([
    ("num", num_tf, num_cols),              #Numerical features
    ("cat", cat_tf, cat_cols)              #Categorical features 
])

In [ ]:
# Fit the pipeline on the data and transform
processed_data = preprocessor.fit_transform(df)

# If you want it back as a DataFrame with column names
import numpy as np

# Get transformed column names
num_features = num_cols
cat_features = cat_cols
all_features = np.concatenate([num_features, cat_features])

processed_df = pd.DataFrame(processed_data, columns=all_features)


ValueError: DataFrame constructor not properly called!

In [8]:
processed_df

NameError: name 'processed_df' is not defined

In [9]:
preprocessor

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [10]:
joblib.dump(preprocessor, "preprocessing_pipeline.pkl")

['preprocessing_pipeline.pkl']

In [11]:
# -----------------------------------------------------------
# Split the data into training and testing sets
# -----------------------------------------------------------

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((960000, 19), (240000, 19), (960000,), (240000,))

In [12]:
# -----------------------------------------------------------
# Import evaluation metrics
# -----------------------------------------------------------

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [13]:
# -----------------------------------------------------------
# mlflow multiple model registry
# -----------------------------------------------------------

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor

models = [
    (
        "Random_Forest_Regressor",
        {"n_estimators": 15, "max_depth": 5},
        RandomForestRegressor(),
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "Gradient_Boosting_Regressor",
        {"n_estimators": 15, "learning_rate": 0.1},
        GradientBoostingRegressor(),
        (x_train, y_train),
        (x_test, y_test)
    ),
    (
        "XGB_Regressor",
        {"n_estimators": 15, "learning_rate": 0.1},
        XGBRegressor(),
        (x_train, y_train),
        (x_test, y_test)
    )
]

In [14]:
reports = []

for model_name, params, model_instance, train_data, test_data in models:
    x_train, y_train = train_data
    x_test, y_test = test_data

    model_instance.set_params(**params)

    pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),   # OrdinalEncoder here
        ("model", model_instance)
    ])

    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    reports.append((model_name, mae, mse, r2))


In [15]:
reports

[('Random_Forest_Regressor',
  654.7148970633698,
  728161.2684261841,
  0.025591712262732824),
 ('Gradient_Boosting_Regressor',
  663.1868111054209,
  736516.0265637141,
  0.014411571373276977),
 ('XGB_Regressor', 652.2488469378153, 721199.9451016366, 0.03490719145008181)]

In [ ]:
# #Train, predict and evaluate each model
# reports = []
# for model_name, params, model_instance, train_data, test_data in models:
#     x_train = train_data[0]
#     y_train = train_data[1]
#     x_test = test_data[0]
#     y_test = test_data[1]
#     #Apply hyperparameters and train the model'
#     model_instance.set_params(**params)
#     model_instance.fit(x_train, y_train)
#     #Make predictions
#     y_pred = model_instance.predict(x_test)

#     #Evaluate the model
#     mae = mean_absolute_error(y_test, y_pred)
#     mse = mean_squared_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)

#     #Store the results
#     reports.append((model_name, mae, mse, r2))
# #Display the evaluation reports
# reports

ValueError: could not convert string to float: 'Male'

In [ ]:
#Model Registration with MLflow
model_name = "Random Forest Regressor"
run_id = input("Enter the run ID")
model_uri = f"runs:/{run_id}/model_name"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, model_name=model_name )

In [ ]:
mlflow.set_experiment("Smart_Premium_Model_Registry")
mlflow.set_tracking_uri("http://127.0.0.1:8080")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics("MAE", mae)
    mlflow.log_metrics("MSE", mse)
    mlflow.log_metrics("R2_Score", r2)

    mlflow.sklearn.log_model(models, "model")    

In [ ]:
mlflow.set_experiment("Smart_Premium_Model_Registry")
mlflow.set_tracking_uri("http://127.0.0.1:8080")    

for i , element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model_instance = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({"MAE": report[i],
                            "MSE": report[i],
                            "R2_Score": report[i]})
        
        if "Random Forest Regressor" in model_name:
            mlflow.RandomForestRegressor.log_model(model_instance, "model")
        elif "Gradient Boosting Regressor" in model_name:
            mlflow.GradientBoostingRegressor.log_model(model_instance, "model")
        elif "XGB Regressor" in model_name:
            mlflow.xgboost.log_model(model_instance, "model")
            

In [ ]:
# -----------------------------------------------------------
# DAGSHUB INSTALLATION 
# -----------------------------------------------------------

# pip3 install dagshub --upgrade                             # Terminal command to install dagshub

# Terminal command to upload file to dagshub
# dagshub upload --bucket udhayakumar240192/smart_premium "file_path"   

# Terminal command to download file from dagshub
# dagshub download --bucket udhayakumar240192/smart_premium "file_path"

Install MLFlow & the DagsHub python client

In [9]:
%pip install -q dagshub mlflow


Note: you may need to restart the kernel to use updated packages.


Use the DagsHub client to setup connection information for MLflow

In [16]:
import dagshub
dagshub.init(repo_owner='udhayakumar240192', repo_name='smart_premium', mlflow=True)

Accessing as udhayakumar240192

Initialized MLflow to track repo "udhayakumar240192/smart_premium"

Repository udhayakumar240192/smart_premium initialized!

In [ ]:
import os
# os.environ["MLFLOW_TRACKING_USERNAME"] = "udhayakumar240192"
# os.environ["MLFLOW_TRACKING_PASSWORD"] = "YOUR_DAGSHUB_TOKEN"

mlflow.set_tracking_uri(
    "https://dagshub.com/udhayakumar240192/smart_premium.mlflow"
)



In [94]:
# -----------------------------------------------------------
# MLflow experiment setup
# -----------------------------------------------------------

mlflow.set_tracking_uri("https://dagshub.com/udhayakumar240192/smart_premium")
mlflow.set_experiment("DS_Smart_Premium_Experiments_2")

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 404 != 200. Response body: '<!DOCTYPE html>
<html lang="en-US">
  <head data-suburl="">
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    <meta http-equiv="X-UA-Compatible" content="IE=edge"/>
    
    
      <meta name="viewport" content="width=device-width, initial-scale=1"/>
    
    <meta name="referrer" content="no-referrer"/>
    <meta name="_csrf" content="3TMwOuQZUyekwvDkU2DyowQ99Xw6MTc2NjY0MzcyNjM2NDMyMDg3OQ"/>
    <meta name="_suburl" content=""/>
    <meta name="_appversion" content="1e79686b5e5b15b3ab7361b2825e6a9562c0f622" />
    <meta name="_bareRepo" content=""/>

    
    
  <script>(function(w, d, s, l, i) {
      w[l] = w[l] || [];
      w[l].push({
        'gtm.start':
          new Date().getTime(), event: 'gtm.js'
      });
      var f = d.getElementsByTagName(s)[0],
        j = d.createElement(s),
        dl = l != 'dataLayer' ? '&l=' 

In [ ]:
import dagshub
dagshub.init(repo_owner='udhayakumar240192', repo_name='smart_premium', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [17]:
mlflow.set_experiment("DS_Smart_Premium")
for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name = model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({"RMSE" : report[1],
                            "MAE" : report[2],
                            "R2" : report[3]
                            })
        
        if "XGB Regressor" in model_name:
            booster = model.get_booster()
            mlflow.xgboost.log_model(booster, "xgboost")

            #mlflow.xgboost.log_model(model, "xgboost")
        else:
            mlflow.sklearn.log_model(model, "sklearn")

2025/12/25 22:19:59 INFO mlflow.tracking.fluent: Experiment with name 'DS_Smart_Premium' does not exist. Creating a new experiment.
2025/12/25 22:20:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random_Forest_Regressor at: https://dagshub.com/udhayakumar240192/smart_premium.mlflow/#/experiments/1/runs/506f389cd0024de29b04ce1cf20d689a
🧪 View experiment at: https://dagshub.com/udhayakumar240192/smart_premium.mlflow/#/experiments/1


2025/12/25 22:20:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Gradient_Boosting_Regressor at: https://dagshub.com/udhayakumar240192/smart_premium.mlflow/#/experiments/1/runs/9b27cda47dc1403d93b91c8b564cc1e4
🧪 View experiment at: https://dagshub.com/udhayakumar240192/smart_premium.mlflow/#/experiments/1


2025/12/25 22:20:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGB_Regressor at: https://dagshub.com/udhayakumar240192/smart_premium.mlflow/#/experiments/1/runs/6a96c7715cf842a19b72f482498ed357
🧪 View experiment at: https://dagshub.com/udhayakumar240192/smart_premium.mlflow/#/experiments/1


In [18]:
#Model Registration with MLflow
model_name = "Random_Forest_Regressor"
run_id = input("Enter the run ID")

model_uri = f"runs:/{run_id}/sklearn"
mlflow.register_model(model_uri, model_name)

Registered model 'Random_Forest_Regressor' already exists. Creating a new version of this model...
2025/12/25 22:23:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random_Forest_Regressor, version 2
Created version '2' of model 'Random_Forest_Regressor'.


<ModelVersion: aliases=[], creation_timestamp=1766681587990, current_stage='None', description='', last_updated_timestamp=1766681587990, name='Random_Forest_Regressor', run_id='506f389cd0024de29b04ce1cf20d689a', run_link='', source='mlflow-artifacts:/c80e92b3fa3c44ca8ca46e670441dff1/506f389cd0024de29b04ce1cf20d689a/artifacts/sklearn', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [19]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

client.transition_model_version_stage(
    name=model_name,
    version=1,            # change to your model version
    stage="Production",
    archive_existing_versions=True
)

C:\Users\Udhaya\AppData\Local\Temp\ipykernel_13016\3971167012.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1766596214106, current_stage='Production', description='', last_updated_timestamp=1766681855997, name='Random_Forest_Regressor', run_id='055f8821c1e349cb9da813e88af14a03', run_link='', source='mlflow-artifacts:/41a88845bcf94f26a2af4bb79c97fa7f/055f8821c1e349cb9da813e88af14a03/artifacts/sklearn', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [20]:
import mlflow.pyfunc

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/Production"
)

In [ ]:
joblib.dump(model, "smart_premium_model.pkl")

['smart_premium_model.pkl']

In [ ]:
#-----------------------------------------------------------
# Load evaluation dataset
#-----------------------------------------------------------

x_evaluate = pd.read_csv(r"D:\project\smart_premium\data\test.csv", index_col="id")
y_evaluate = pd.read_csv(r"D:\project\smart_premium\data\sample_submission.csv", index_col="id")

In [23]:
preprocess_pipeline = joblib.load(r"D:\project\smart_premium\preprocessing_pipeline.pkl")
x_evaluate_preprocessed = preprocess_pipeline.transform(x_evaluate)
x_evaluate_preprocessed

array([[-0.97835937, -0.95219826,  1.47368116, ...,  1.        ,
         3.        ,  2.        ],
       [-0.75504563,  2.96109258, -0.00668075, ...,  1.        ,
         2.        ,  0.        ],
       [ 0.43596095, -0.48464412, -1.48704266, ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [-1.12723519,  0.08741541, -1.48704266, ...,  0.        ,
         1.        ,  0.        ],
       [-0.5317319 ,  0.41899233,  0.7335002 , ...,  0.        ,
         3.        ,  1.        ],
       [-1.2016731 , -0.23948027,  0.7335002 , ...,  0.        ,
         2.        ,  2.        ]], shape=(800000, 19))

In [24]:
from sklearn.metrics import root_mean_squared_error


rfr_model = mlflow.sklearn.load_model(model_uri= 'models:/Random_Forest_Regressor/Production')

y_pred = rfr_model.predict(x_evaluate_preprocessed)
print("MAE:", mean_absolute_error(y_evaluate,y_pred))
print("RMSE:", root_mean_squared_error(y_evaluate,y_pred))
print("R2 :", r2_score(y_evaluate,y_pred))

d:\project\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


MAE: 1102.544203308085
RMSE: 1102.544214924142
R2 : 0.0
